In [ ]:

from collections import OrderedDict
import datetime
import pymongo
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#from plotly import __version__
#print(__version__) # requires version >= 1.9.0

In [ ]:
class Model:
    collection = None

    def __init__(self):
        self.connect_db()

    def connect_db(self):
        client = pymongo.MongoClient()
        client = pymongo.MongoClient('localhost', 27017)
        db = client.mdb
        #mdb is the database created
        print("Connected to the database.")
#         self.collection = db.measurements
        self.collection = db.measure2

        # Display amount of database items
        cursor = self.collection.find()
        print("The database contains", cursor.count(), "measurements.")

    def search_recursive(self, date_start, date_end, key):
        ss = date_start.strftime('%Y-%m-%d')
        en = date_end.strftime('%Y-%m-%d')

        # Split search field into individual search terms
        search_keys = key.split()

        # Create iterator for all database entries
        # in the selected time range
        cursor = self.collection.find({
#             'Datum nach Pfad': {
            'DataPath':{
                '$gte': ss,
                '$lte': en
            }
        })

        # Do a full text search in the found entries
        found_items = []
        if cursor.count() > 0:
            # Iterate over all database entries
            for item in cursor:
                match = False
                # Check if all search_keys are found in the item
                for search_key in search_keys:
                    # Make all search queries case insensitive
                    if search_key.lower() in str(item).lower():
                        match = True
                    else:
                        match = False
                    if match == False:
                        break
                # All search keys have been found
                if match == True:
                    found_items.append(item)

        print("{} items found.".format(len(found_items)))

        return found_items

In [ ]:
m = Model()

In [ ]:
# Q1,How many measurements did the people do (absolute and percentage)
date_start = datetime.date(2015, 9, 7)
date_end = datetime.date.today()

key = 'We'
results = m.search_recursive(date_start, date_end, key)

len(results)

names = {}
for result in results:
    if key in result:
        name = result[key]
        if not name in names:
            names[name] = 1
        else:
            names[name] += 1
    
names
names_sorted_by_value = OrderedDict(sorted(names.items(), key=lambda x: x[1],reverse= True))

In [ ]:
names_str = []
names_count = []
for name in names_sorted_by_value:
    names_str.append(name)
    names_count.append(names_sorted_by_value[name])

data = [go.Bar(
            x=names_str,
            y=names_count
    )]

py.iplot(data, filename='basic-bar')

# plot the percentage of the person who did the lab 
percentage = go.Pie(labels=names_str,values=names_count)
py.iplot([percentage],filename='percentage of names')

In [ ]:
# Q7 Show the temperature 
import re

date_start = datetime.date(2015, 9, 7)
date_end = datetime.date.today()

key = 'tem'
results = m.search_recursive(date_start, date_end, key)

len(results)

speks= {}
for result in results:
    if key in result:
        spek = result['DataPath']
        if not spek.strip() in speks:
            numbers =re.findall('\d+\.\d+|\d+',result[key])
            if len(numbers)>0:
                speks[spek]=numbers[0]
                
speks


speks_sorted_by_value = OrderedDict(sorted(speks.items(), key=lambda x: x[0]))
speks_sorted_by_value

speks_str = []
speks_count = []
for spek  in speks_sorted_by_value:
    speks_str.append(spek)
    speks_count.append(speks_sorted_by_value[spek])

trace = go.Scatter(
    x = speks_str,
    y = speks_count,
    name = 'Low 2014',
    mode = 'markers',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,)
)

data = [trace]
layout = dict(title = ' Temperatures',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Temperature (degrees)'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')




In [ ]:
# Q8--	List all substances


date_start = datetime.date(2015, 9, 7)
date_end = datetime.date.today()

key = 'dat'
results = m.search_recursive(date_start, date_end, key)

len(results)

speks = {}
for result in results:
    if key in result:
        spek = result[key]
        for rawdata in spek:
            if not rawdata.strip() in speks:
                speks[rawdata.strip()] = 1
            else:
                speks[rawdata.strip()] += 1
    
# for ii in speks:
#     print(ii,speks[ii])
print("we use",len(speks),"different cuvetts lengths")

speks_sorted_by_value = OrderedDict(sorted(speks.items(), key=lambda x: x[1],reverse= True))
speks_sorted_by_value

speks_str = []
speks_count = []
for spek  in speks_sorted_by_value:

    speks_str.append(spek)
    speks_count.append(speks_sorted_by_value[spek])

data = [go.Bar(
            x=speks_str,
            y=speks_count
    )]

py.iplot(data, filename='spek_count')

percentage = [go.Bar(
            x=speks_str,
            y=list(map(lambda x: x*1.0/len(speks), speks_count))
    )]

py.iplot(percentage,filename='percentage of speks')

